videos is a list of dict objects, each with the following keys:

"videos": {
    "id": int,
    "video_id": str,
    "category": int,
    "url": str,
    "start time": float,
    "end time": float,
    "split": str
  }

  "sentences": {
    "sen_id": int,
    "video_id": str,
    "caption": str
  }

In [1]:

import numpy as np
import os
import math
from skimage.io import imread, imshow
from skimage.transform import resize
import matplotlib.pyplot as plt 
import cv2
import pytube
import tensorflow as tf
import matplotlib.pyplot as plt
import requests
from os import path

In [2]:
def getutl(a="youtubeurl"):
          a=str(a)
          for i in range(len(a)):
              if a[i]=="v":
                idy=a[i+2:]
                break
          b=requests.get("http://img.youtube.com/vi/{}/mqdefault.jpg".format(idy))
          if b.status_code==200:
              return True
          else:
              return False


In [3]:
import json
filename='videodatainfo_2017.json'
opened= open(filename, 'r')
trainData=json.load(opened)
opened.close()

In [4]:
videos=trainData["videos"]
captions=trainData["sentences"]
newCap={}
for sent in captions:
    newCap[sent["video_id"]]=[]
for sent in captions:
    newCap[sent["video_id"]].append(sent["caption"])
captions=newCap



In [5]:
inputShape=(299,299,3)
image_model=tf.keras.applications.xception.Xception(weights='imagenet',include_top=False, input_shape=inputShape, pooling="avg")
new_input = image_model.input
hidden_layer = image_model.layers[-1].output

model = tf.keras.Model(new_input, hidden_layer)


In [7]:
class preprocess:
  def __init__(self, video):
    self.url=video["url"]
    self.id=video["video_id"]
    self.category=video["category"]
    self.start=video["start time"]
    self.end=video["end time"]
    self.split=video["split"]
    self.path="videos"
    self.images=[]
    self.execute()
  
  def getVideo(self):
    request = requests.get(self.url)

    if getutl(self.url):
      if (not path.exists(self.path+'/'+self.id+'.mp4')):
        print(self.id)
        self.video=pytube.YouTube(self.url)
        self.video=self.video.streams.first()
        self.video.download(self.path, self.id)
      self.path+='/'+self.id+'.mp4'
      return True
    else:
      return False


  def getFrame(self, sec):
      self.video.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
      hasFrames,image = self.video.read()
      if hasFrames:
        self.images.append(image)
      return hasFrames
  
  def vidToFrame(self):
    self.video = cv2.VideoCapture(self.path)
    start_time_ms = self.start
    frameRate = (self.end-self.start)/75 
    count=0
    success = True
    while success and self.video.get(cv2.CAP_PROP_POS_MSEC) < self.start*1000:
      success, image = self.video.read()

    while success and self.video.get(cv2.CAP_PROP_POS_MSEC) <= self.end*1000:
      success, image = self.video.read()
      count += 1

      if success:
        self.images.append(image)
    
    scaleFactor = 80
    skipFactor = len(self.images)//scaleFactor
    frames = [self.images[i] for i in range(0, len(self.images), skipFactor)][:scaleFactor]
    self.images = frames
    
  def execute(self):
    if self.getVideo():
      self.vidToFrame()
      self.video=0

In [8]:
features={}
done = 1
for video in videos:
    try:
        try:
            frames=preprocess(video)
            if len(frames.images)!=0:
                features[frames.id]=[]
                c = 0
                frames_preprocessed = []
                for image in frames.images[:80]:
                    inputShape=(inputShape[0],inputShape[1])
                    img=cv2.resize(image, inputShape)
                    img=tf.keras.preprocessing.image.img_to_array(img)
                    img=tf.keras.applications.xception.preprocess_input(img)
                    frames_preprocessed.append(img)
                    c+=1
                frames_tensor = np.array(frames_preprocessed)
                video_features = model(frames_tensor)
                if(frames_tensor.shape[0]==80):
                    video_features = np.reshape(video_features, (video_features.shape[0], -1))
                    np.save("features/"+frames.id, video_features)
                    print(done)
                    done+=1
        except ("RegexMatchError"):
            pass
    except NameError:
        pass
    except TypeError:
        pass

      



4707
4708
4709
4710
4711
4712
4713
4714
4715
4716
4717
4718
4719
4720
4721
4722
4723
4724
4725
4726
4727
4728
4729
4730
4731
4732
4733
4734
4735
4736
4737
4738
4739
4740
4741
4742
4743
4744
4745
4746
4747
4748
4749
4750
4751
4752
4753
4754
4755
4756
4757
4758
4759
4760
4761
video6173
4762
4763
4764
4765
4766
4767
4768
4769
4770
4771
4772
4773
4774
4775
4776
4777
4778
4779
4780
4781
4782
4783
4784
4785
4786
4787
4788
4789
4790
4791
4792
4793
4794
4795
4796
4797
4798
4799
4800
4801
4802
4803
4804
4805
4806
4807
4808
4809
4810
4811
4812
4813
4814
4815
4816
4817
4818
4819
4820
4821
4822
4823
4824
4825
4826
4827
video6250
4828
4829
4830
4831
4832
4833
4834
4835
4836
4837
4838
4839
4840
video6266
4841
4842
4843
4844
4845
4846
4847
4848
4849
4850
4851
4852
4853
4854
4855
4856
4857
4858
4859
4860
4861
4862
4863
4864
4865
4866
4867
4868
4869
4870
4871
4872
4873
4874
4875
4876
4877
4878
4879
4880
4881
4882
4883
4884
4885
4886
video6325
4887
4888
4889
4890
4891
4892
4893
4894
4895
4896
4897
4898
